In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from datetime import datetime, timedelta
import pandas_datareader as pdr

# Download stock data
stock_data = yf.download('NVDA', start='2023-01-01', end='2024-07-01')

# Add earnings reports data
nvda = yf.Ticker('NVDA')
earnings = nvda.earnings_dates
earnings.reset_index(inplace=True)
earnings = earnings.rename(columns={"index": "Date"})
earnings['Date'] = pd.to_datetime(earnings['Date'])
stock_data = stock_data.merge(earnings[['Date']], how='left', on='Date')
stock_data['Earnings'] = stock_data['Date'].notna().astype(int)

# Add economic indicators data: GDP
gdp_data = pdr.get_data_fred('GDP', start=datetime(2023, 1, 1), end=datetime(2024, 7, 1))
gdp_data = gdp_data.rename(columns={"GDP": "GDP_Quarterly"})
gdp_data = gdp_data.resample('D').ffill().reset_index()  # Fill daily with forward fill
stock_data = stock_data.reset_index().merge(gdp_data, how='left', on='Date').set_index('Date')

# Create additional features
stock_data['MA_20'] = stock_data['Close'].rolling(window=20).mean()  # 20-day moving average
stock_data['MA_50'] = stock_data['Close'].rolling(window=50).mean()  # 50-day moving average

# Create a binary target variable for price increase
stock_data['Price_Increase'] = (stock_data['Close'].shift(-1) > stock_data['Close']).astype(int)
stock_data.dropna(inplace=True)  # Drop NaN values created by moving averages

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(stock_data[['Close', 'Open', 'High', 'Low', 'Volume', 'MA_20', 'MA_50', 'Earnings', 'GDP_Quarterly']].values)

# Create dataset
def create_dataset(data, time_step):
    X, y_price, y_increase = [], [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step)])
        y_price.append(data[i + time_step, 0])  # Predicting 'Close' price
        y_increase.append(data[i + time_step, -1])  # Predicting 'Price_Increase'
    return np.array(X), np.array(y_price), np.array(y_increase)

time_step = 60
X, y_price, y_increase = create_dataset(scaled_data, time_step)

# Check if the dataset is empty
if X.size == 0 or y_price.size == 0 or y_increase.size == 0:
    raise ValueError("Not enough data to create the dataset with the given time_step. Please use a smaller time_step.")

# Split data into train and test sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_price_train, y_price_test = y_price[:train_size], y_price[train_size:]
y_increase_train, y_increase_test = y_increase[:train_size], y_increase[train_size:]

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=64, return_sequences=True, input_shape=(time_step, X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=64))
model.add(Dropout(0.2))
model.add(Dense(units=64))
model.add(Dense(units=2))  # Two outputs: price and increase/decrease

# Compile and fit the model
model.compile(optimizer='adam', loss=['mean_squared_error', 'binary_crossentropy'], metrics=['accuracy'])
model.fit(X_train, [y_price_train, y_increase_train], epochs=50, batch_size=64, validation_data=(X_test, [y_price_test, y_increase_test]))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, [y_price_test, y_increase_test])
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

# Predict the test data
predictions_price, predictions_increase = model.predict(X_test)
predictions_price = scaler.inverse_transform(np.concatenate((predictions_price, np.zeros((predictions_price.shape[0], 8))), axis=1))[:, 0]
predictions_increase = (predictions_increase > 0.5).astype(int)

# Plot the original and predicted data
original_data = stock_data['Close'].values
predicted_data = np.empty_like(original_data)
predicted_data[:] = np.nan
predicted_data[-len(predictions_price):] = predictions_price

plt.plot(original_data, label='Original Data')
plt.plot(predicted_data, label='Predicted Data')
plt.legend()
plt.show()

# Predict the next week's stock prices
X_input = scaled_data[-time_step:].reshape(1, time_step, X.shape[2])
predictions = []

for _ in range(7):
    next_prediction_price, next_prediction_increase = model.predict(X_input)
    predictions.append(next_prediction_price[0, 0])
    next_input = np.concatenate((next_prediction_price, np.zeros((1, 8))), axis=1)
    X_input = np.append(X_input[:, 1:, :], next_input.reshape(1, 1, X.shape[2]), axis=1)

predicted_prices = scaler.inverse_transform(np.concatenate((np.array(predictions).reshape(-1, 1), np.zeros((len(predictions), 8))), axis=1))[:, 0]

# Generate dates for the next week
last_date = stock_data.index[-1]
dates = [last_date + timedelta(days=i) for i in range(1, 8)]

# Plotting the predictions
plt.figure(figsize=(14, 7))
plt.plot(dates, predicted_prices, marker='o', linestyle='-', color='orange', label='Predicted Data')
plt.xlabel('Date')
plt.ylabel('Stock Price')
plt.title('Predicted Stock Prices for the Next Week')
plt.legend()
plt.grid(True)
plt.show()

# Print the predicted opening and closing prices for each day
print("Predicted Opening and Closing Prices for the Next Week:")
for i, date in enumerate(dates):
    print(f"{date.date()}: Open: {predicted_prices[i]:.2f}, Close: {predicted_prices[i]:.2f}")
